# Use R Shiny app to create SIR model with `ibm-watsonx-ai`

This notebook contains steps and code to demonstrate support of external R Shiny application code with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in PyPI repository. 

Some familiarity with Python is helpful. This notebook uses Python 3.12.

## Learning goals

The learning goals of this notebook are:

-  Persist a R Shiny app in in watsonx.ai repository.
-  Deploy application for online scoring using client library.
-  Score sample records using client library.


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Application upload](#upload)
3. [Web service creation](#deploy)
4. [Clean up](#cleanup)
5. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U wget | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [ ]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="upload"></a>
## 2. Application upload

In this section you will learn how to store downloaded application as an asset.

#### Download an R Shiny app

In [ ]:
import os
import wget

filename = "app.R.zip"
url = "https://github.com/IBM/watsonx-ai-samples/raw/master/cpd5.2/models/r_shiny/app.R.zip"
if not os.path.isfile(filename):
    filename = wget.download(url)

#### Upload application as data asset

In [8]:
shiny_details = client.shiny.store(
    meta_props={
        client.shiny.ConfigurationMetaNames.NAME: "R Shiny app - SIR",
    },
    file_path=filename,
)
shiny_asset_id = client.shiny.get_id(shiny_details)

Creating Shiny asset...
SUCCESS


You can now see that the application is saved in watsonx.ai

In [9]:
client.shiny.list(limit=5)

,NAME,ASSET_TYPE,ASSET_ID
0,R Shiny app - SIR,shiny_asset,dd053eb9-b8fc-4e98-9fad-3bae97f8fc37


<a id="deploy"></a>
## 3. Create online deployment
You can use commands bellow to create online deployment for stored application (web service).

In [10]:
deployment = client.deployments.create(
    artifact_id=shiny_asset_id,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "Deployment R Shiny's SIR",
        client.deployments.ConfigurationMetaNames.R_SHINY: {
            "authentication": "anyone_with_url"
        },
        client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
            "name": "S",
            "num_nodes": 1,
        },
    },
)



######################################################################################

Synchronous deployment creation for id: 'dd053eb9-b8fc-4e98-9fad-3bae97f8fc37' started

######################################################################################


initializing
Note: rshiny_url is deprecated and will be removed in a future release. Use serving_urls instead.
.........
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='145cf352-3d40-40f5-a460-adb6a1a4ac4c'
-----------------------------------------------------------------------------------------------




#### Get deployments details

In [ ]:
deployment_id = client.deployments.get_id(deployment)
client.deployments.get_details(deployment_id)

<a id="cleanup"></a>
## 4. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 5. Summary and next steps     

You successfully completed this notebook! You learned how to use Watson Machine Learning for SPSS model deployment and scoring. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.